In [12]:
#@title imports and installing
!pip install torch torchvision


In [ ]:
# Install dependencies
!pip install torch==1.10.0+cu102 torchvision==0.11.1+cu102 torchaudio==0.10.0+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

# Install Detectron2
!pip install git+https://github.com/facebookresearch/detectron2.git


In [ ]:
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html


In [ ]:
!pip install torch==1.10.0+cu102 torchvision==0.11.1+cu102 torchaudio==0.10.0+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.10/index.html


In [18]:
# import json

# with open("/content/drive/MyDrive/Task_4/train/_annotations.coco.json") as f:
#     json.load(f)

# with open("/content/drive/MyDrive/Task_4/valid/_annotations.coco.json") as f:
#     json.load(f)
import string

with open("/content/drive/MyDrive/Task_4/train/_annotations.coco.json", "r", encoding="utf-8") as f:
    content = f.read()
    content = ''.join(filter(lambda x: x in string.printable, content))

with open("/content/drive/MyDrive/Task_4/train/_annotations_cleaned.coco.json", "w", encoding="utf-8") as f:
    f.write(content)


In [ ]:
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2 import model_zoo
import os
import cv2
from detectron2.utils.visualizer import Visualizer
import random
import json
import string

def clean_json(json_path, cleaned_json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
            cleaned_data = json.dumps(data, ensure_ascii=False, separators=(',', ':'))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in {json_path}: {e}")
            return

    with open(cleaned_json_path, "w", encoding="utf-8") as f:
        f.write(cleaned_data)

# Path to the original JSON files
train_json_path = "/content/drive/MyDrive/Task_4/train/_annotations.coco.json"
val_json_path = "/content/drive/MyDrive/Task_4/valid/_annotations.coco.json"

# Clean the training JSON file
cleaned_train_json_path = "/content/drive/MyDrive/Task_4/train/_annotations_cleaned.coco.json"
clean_json(train_json_path, cleaned_train_json_path)

# Clean the validation JSON file
cleaned_val_json_path = "/content/drive/MyDrive/Task_4/valid/_annotations_cleaned.coco.json"
clean_json(val_json_path, cleaned_val_json_path)

# Unregister the 'dentistry_dataset_train' dataset if it's already registered
if "dentistry_dataset_train_v2" in DatasetCatalog:
    DatasetCatalog.remove("dentistry_dataset_train_v2")
    MetadataCatalog.remove("dentistry_dataset_train_v2")

# Unregister the 'dentistry_dataset_val' dataset if it's already registered
if "dentistry_dataset_val_v2" in DatasetCatalog:
    DatasetCatalog.remove("dentistry_dataset_val_v2")
    MetadataCatalog.remove("dentistry_dataset_val_v2")

# Register the Roboflow dataset with the cleaned JSON files
register_coco_instances("dentistry_dataset_train_v2", {}, cleaned_train_json_path, "/content/drive/MyDrive/Task_4/train")
register_coco_instances("dentistry_dataset_val_v2", {}, cleaned_val_json_path, "/content/drive/MyDrive/Task_4/valid")

# Configuration and training
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("dentistry_dataset_train_v2",)
cfg.DATASETS.TEST = ("dentistry_dataset_val_v2",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 300
cfg.SOLVER.STEPS = []  # Adjust this based on your dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # Number of classes (including background)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Perform inference and visualization
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Adjust as needed
predictor = DefaultPredictor(cfg)

# Visualization
dataset_dicts = DatasetCatalog.get("dentistry_dataset_val_v2")
for d in random.sample(dataset_dicts, 3):  # Change 3 to the number of images you want to visualize
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get("dentistry_dataset_val_v2"), scale=1.2)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("Visualization", v.get_image()[:, :, ::-1])
    cv2.waitKey(0)
